In [136]:
#! /usr/bin/python  
import numpy as np  
import xgboost as xgb
import pandas as pd
  

In [227]:
# label need to be 0 to num_class -1  
# if col 33 is '?' let it be 1 else 0, col 34 substract 1  
data = pd.read_csv('/Users/tinggao/Documents/CU course/YieldStreet/wd/trydt.csv')
# data = data.drop(list(data.columns[[0,1,2,9]]),axis=1)
data.head(5)

,offering_launch_date,offering_size,minimum_investment_amount,maximum_investment_amount,investor_yield,term,seconds
0,1295,600000.0,5000.0,240000.0,13.0,36,7153223
1,1218,700000.0,5000.0,280000.0,13.0,36,1350059
2,1213,270300.0,5000.0,270300.0,20.0,18,1209600
3,1195,800000.0,5000.0,25000.0,13.0,36,77126431
4,1175,250000.0,5000.0,260000.0,15.0,19,1555200


In [228]:
data.seconds = data.seconds.map(lambda x: (int(x>1e+07) + int(x>2e+07) + int(x>3e+07)+int(x>4e+07)+int(x>5e+07)))
data.head(5)

,offering_launch_date,offering_size,minimum_investment_amount,maximum_investment_amount,investor_yield,term,seconds
0,1295,600000.0,5000.0,240000.0,13.0,36,0
1,1218,700000.0,5000.0,280000.0,13.0,36,0
2,1213,270300.0,5000.0,270300.0,20.0,18,0
3,1195,800000.0,5000.0,25000.0,13.0,36,5
4,1175,250000.0,5000.0,260000.0,15.0,19,0


In [172]:
sz = data.shape

In [ ]:
from sklearn.model_selection import train_test_split


In [232]:
# train = data.iloc[:int(sz[0] * 0.7), :] # take row 1-256 as training set  
# test = data.iloc[int(sz[0] * 0.7):, :]  # take row 257-366 as testing set  
# train_X = train.iloc[:,0:5]  
# train_Y = train.iloc[:,6]  
# test_X = test.iloc[:,0:5]
# test_Y = test.iloc[:,6] 

train_X,test_X,train_Y,test_Y = train_test_split(data.iloc[:,0:5], data.iloc[:,6], test_size=0.33, random_state=42)


test_X.reset_index(drop=True,inplace=True)
test_Y.reset_index(drop=True,inplace=True)

  
xg_train = xgb.DMatrix(train_X, label=train_Y)  
xg_test = xgb.DMatrix(test_X, label=test_Y)  

In [322]:
# setup parameters for xgboost  
param = {}  
# use softmax multi-class classification  
param['objective'] = 'multi:softmax'  
# scale weight of positive examples  
param['eta'] = 0.1  
param['max_depth'] = 6  
param['silent'] = 1  
param['nthread'] = 4  
param['num_class'] = 6 
param['max_depth'] = 25
# param['eval_metric']='mlogloss'
param['eval_metric']='merror'
num_round = 9

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]  
bst = xgb.train(param, xg_train, num_round, watchlist );

[0]	train-merror:0.194444	test-merror:0.421053
[1]	train-merror:0.194444	test-merror:0.421053
[2]	train-merror:0.194444	test-merror:0.421053
[3]	train-merror:0.194444	test-merror:0.421053
[4]	train-merror:0.138889	test-merror:0.421053
[5]	train-merror:0.138889	test-merror:0.421053
[6]	train-merror:0.138889	test-merror:0.421053
[7]	train-merror:0.138889	test-merror:0.421053
[8]	train-merror:0.111111	test-merror:0.368421


In [323]:
pred = bst.predict(xg_test)
pred

array([2., 0., 2., 2., 3., 0., 1., 2., 2., 2., 0., 0., 2., 2., 4., 4., 0.,
       0., 2.], dtype=float32)

In [324]:
list(test_Y)

[2, 0, 0, 4, 3, 0, 1, 2, 0, 4, 0, 5, 2, 2, 1, 3, 0, 0, 2]

In [325]:
# get prediction  
pred = bst.predict(xg_test);  
# pred = np.argmax(pred, axis=1)
np.sum(pred == test_Y) / len(test_Y)

0.631578947368421

In [318]:
# np.array(np.argmax(pred,axis=1))

In [319]:
# np.array(train_Y)

**softprob**

In [258]:
# do the same thing again, but output probabilities  
param['objective'] = 'multi:softprob'  
bst = xgb.train(param, xg_train, num_round, watchlist );  
# Note: this convention has been changed since xgboost-unity  
# get prediction, this is in 1D array, need reshape to (ndata, nclass)  
yprob = bst.predict( xg_test ).reshape( test_Y.shape[0], 6 )  
ylabel = np.argmax(yprob, axis=1)  # return the index of the biggest pro  
  
print ('predicting, classification error=%f' % (sum( int(ylabel[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) )) 

[0]	train-merror:0.194444	test-merror:0.421053
[1]	train-merror:0.194444	test-merror:0.421053
[2]	train-merror:0.194444	test-merror:0.421053
[3]	train-merror:0.194444	test-merror:0.421053
[4]	train-merror:0.138889	test-merror:0.421053
[5]	train-merror:0.138889	test-merror:0.421053
[6]	train-merror:0.138889	test-merror:0.421053
[7]	train-merror:0.138889	test-merror:0.421053
[8]	train-merror:0.111111	test-merror:0.368421
[9]	train-merror:0.111111	test-merror:0.421053
[10]	train-merror:0.111111	test-merror:0.421053
[11]	train-merror:0.111111	test-merror:0.421053
[12]	train-merror:0.083333	test-merror:0.421053
[13]	train-merror:0.083333	test-merror:0.421053
[14]	train-merror:0.083333	test-merror:0.421053
[15]	train-merror:0.083333	test-merror:0.421053
[16]	train-merror:0.083333	test-merror:0.421053
[17]	train-merror:0.055556	test-merror:0.421053
[18]	train-merror:0.055556	test-merror:0.421053
[19]	train-merror:0.055556	test-merror:0.421053
[20]	train-merror:0.055556	test-merror:0.421053
[2

In [261]:
ylabel = np.argmax(yprob, axis=1)
ylabel

array([2, 0, 2, 2, 3, 0, 1, 2, 2, 2, 0, 0, 2, 0, 4, 4, 0, 0, 2])

In [263]:
list(test_Y)

[2, 0, 0, 4, 3, 0, 1, 2, 0, 4, 0, 5, 2, 2, 1, 3, 0, 0, 2]

In [264]:
ylabel == list(test_Y)

array([ True,  True, False, False,  True,  True,  True,  True, False,
       False,  True, False,  True, False, False, False,  True,  True,
        True])

In [265]:
len(test_Y)

19

In [268]:
11/19

0.5789473684210527

In [327]:
xgb.plot_importance(bst)